In [1]:
import pandas as pd 
import numpy as np
from sqlalchemy import create_engine
import psycopg2
import os

In [24]:
from dotenv import load_dotenv
load_dotenv()

DATABASE = os.getenv('DATABASE')
HOST = os.getenv('HOST')
USER = os.getenv('USER')
PASSWORD = os.getenv('PASSWORD')
PORT = os.getenv('PORT')


## Extraction et mise en forme des données 

In [2]:
df = pd.read_csv('../datas/data_sport.csv', delimiter=',',on_bad_lines='skip', decimal='.')
df.head()

,id,Date,Sport,Séance,Exercice,Durée,poid,Nbr_de_repetitions,Commentaire
0,0,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN
1,1,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN
2,2,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN
3,3,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,50.0,10.0,NaN
4,4,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,50.0,9.0,NaN


In [3]:
def clean_data(x): 
    x = str(x)
    x = x.replace(',','.')
    x = x.replace('\xa0', '')
    return float(x)



In [5]:

df["poid"] = df["poid"].apply(clean_data) 
df["poid"] = df["poid"].astype("float")


In [ ]:
df["Date"] =  pd.to_datetime(df["Date"])


In [6]:
df["Nbr_de_repetitions"] = df["Nbr_de_repetitions"].replace('max',5 )
df["Nbr_de_repetitions"] = df["Nbr_de_repetitions"].apply(clean_data) 
df["Nbr_de_repetitions"] = df["Nbr_de_repetitions"].astype("float")

In [8]:
df["Durée (min)"] = df["Durée"].replace("Nan", np.nan)
df["Durée (min)"] = df["Durée"].replace("7,2", 7.2)
df["Durée"] = df["Durée"].astype("float")


In [9]:
df = df[["Date","Sport","Séance","Exercice","Durée","poid","Nbr_de_repetitions",'Commentaire']]

In [10]:
df["id"] = df.index

/var/folders/p5/j7xh751d45s4z0kzc_3gql0h0000gn/T/ipykernel_60174/2793045622.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["id"] = df.index


## Remplissage de la table musculation_exercices

In [30]:
df_musculation_exercices = df.loc[df["Sport"]=="Musculation"]
df_musculation_exercices = df_musculation_exercices[["Séance","Exercice"]]
df_musculation_exercices = df_musculation_exercices.rename({"Séance":"muscle_area","Exercice":"musculation_exercice_name"},axis=1)
df_musculation_exercices = df_musculation_exercices.drop_duplicates(subset=["muscle_area","musculation_exercice_name"])

In [31]:
df_musculation_exercices.shape[0]
df_musculation_exercices.head()

,muscle_area,musculation_exercice_name
0,Pectoraux,Développé Couché
6,Pectoraux,Ecartés Poulie
10,Dos,Lat Pull
17,Dos,Seated Row Prise Verticale
23,Dos,Biceps Curls


In [32]:
connection_string = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

table_name ="musculation_exercices"
df_musculation_exercices.to_sql(table_name, engine, if_exists='replace', index=False)


97

In [33]:
## Remplissage de la table musculation_rows

In [35]:
df_musculation = df.loc[df["Sport"]=="Musculation"]
df_musculation = df_musculation.drop("Durée",axis=1)

In [38]:
df_musculation =df.rename(columns={"Date":"date_seance","Séance":"MuscleArea","Exercice":"Exercice","poid":"Poid","Nbr_de_repetitions":"NbrRepetition","Commentaire":"Comments","id":"SportSeanceId"})
df_musculation

,date_seance,Sport,MuscleArea,Exercice,Durée,Poid,NbrRepetition,Comments,SportSeanceId
0,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,0
1,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,1
2,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,60.0,6.0,NaN,2
3,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,50.0,10.0,NaN,3
4,2024-02-19,Musculation,Pectoraux,Développé Couché,NaN,50.0,9.0,NaN,4
...,...,...,...,...,...,...,...,...,...
637,2024-05-27,Musculation,Pectoraux,Pec Fly,NaN,66.0,12.0,NaN,637
638,2024-05-27,Musculation,Pectoraux,Pec Fly,NaN,66.0,12.0,NaN,638
639,2024-05-27,Musculation,Pectoraux,Pec Fly,NaN,66.0,12.0,NaN,639
640,2024-05-27,Musculation,Pectoraux,Pec Fly,NaN,66.0,12.0,NaN,640


In [39]:
df_musculation = df_musculation.drop(["Sport","Durée"], axis=1)

In [40]:
df_musculation = df_musculation.dropna(subset=["Poid","NbrRepetition"])

In [41]:
connection_string = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

table_name ="musculation_rows"
df_musculation.to_sql(table_name, engine, if_exists='replace', index=False)


599

## Remplissage de la table sport_rows

In [49]:
df_sport = df.loc[df["Sport"]!="Musculation"]

In [50]:
df_sport.columns

Index(['Date', 'Sport', 'Séance', 'Exercice', 'Durée', 'poid',
       'Nbr_de_repetitions', 'Commentaire', 'id'],
      dtype='object')

In [58]:
df_sport.columns

Index(['date_seance', 'Sport', 'Séance', 'Exercice', 'Durée', 'poid',
       'Nbr_de_repetitions', 'Commentaire', 'id'],
      dtype='object')

In [57]:
df_sport = df_sport.rename({"Date":"date_seance","Sport":"Sport","Durée":"Durée","Commentaire":"Commentaire","id":"id"},axis=1)

In [60]:
df_sport.drop(["Séance","poid","Nbr_de_repetitions"], axis=1)

,date_seance,Sport,Exercice,Durée,Commentaire,id
26,2024-02-24,Cardio,Vélo Elliptique,30.0,NaN,26
27,2024-02-24,Cardio,Cross Trainning,30.0,NaN,27
28,2024-02-24,Cardio,Tennis,60.0,NaN,28
95,2024-02-03,Cardio,Cross Trainning,NaN,20 pompes \n20 ballons mur en équilibres \n12 ...,95
143,2024-04-03,Cardio,Tennis,60.0,NaN,143
159,2024-07-03,Cardio,Vélo Elliptique,25.0,NaN,159
160,2024-07-03,Cardio,NaN,NaN,Pompes\nEpaules\nAbdo allongée jambes levée \n...,160
188,2024-10-03,Cardio,Tennis,60.0,NaN,188
225,2024-03-24,Cardio,Tennis,60.0,NaN,225
226,2024-03-25,Cardio,Vélo Elliptique,30.0,"intensité 13 calorie 381 distances 2,72 km",226


In [61]:
connection_string = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

table_name ="sport_rows"
df_sport.to_sql(table_name, engine, if_exists='replace', index=False)


24

## Remplissage de la table  sport_exercices 

In [63]:
df_sport = df.loc[df["Sport"]!="Musculation"]

In [65]:
df_sport = df_sport[["id","Exercice"]]

In [68]:
df_sport = df_sport.rename({"id":"id","Exercice":"name"},axis=1)

In [70]:
df_sport = df_sport.drop_duplicates(subset='name')

In [72]:
df_sport = df_sport.dropna()

In [73]:
df_sport

,id,name
26,26,Vélo Elliptique
27,27,Cross Trainning
28,28,Tennis
311,311,Course sur Tapis


In [74]:
connection_string = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

table_name ="sport_exercices"
df_sport.to_sql(table_name, engine, if_exists='replace', index=False)


4